In [1]:
import sys
sys.path.append('../')

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import os
import torch
import warnings
from scipy.optimize import minimize
from torch.utils.data import DataLoader

from src.model import SindyModel
from src.main import get_args
from src.dataloader import ScrewdrivingDataset

In [3]:
ARGS = get_args()
data_dir = os.path.join('..', 'out', 'sindy-data')
model_dir = os.path.join('..', 'out', 'sindy-model-out', 'checkpoints')
out_dir = os.path.join('..', 'out', 'sindy-out-processed')
os.makedirs(out_dir, exist_ok=True)
model_checkpoints = sorted(filter(lambda f: f.endswith('.pth'), os.listdir(model_dir)))
print(f'found {len(model_checkpoints)} models')

found 1441 models


In [4]:
%matplotlib inline
plt.ioff()

In [5]:
def signal_error(params, reference_signal, target_signal):
    adjusted_signal = adjust_signal(target_signal, *params)
    return np.sum((reference_signal - adjusted_signal) ** 2)

def adjust_signal(target_signal, mean_adjustment, time_shift):
    shifted_target_signal = np.interp(np.arange(len(target_signal)), np.arange(len(target_signal)) - time_shift, target_signal)
    return shifted_target_signal + mean_adjustment

In [6]:
def plot_signals(
    pred_signals, 
    adjusted_signals, 
    reference_signals,
    model_name,
    show_plot=False,
    title="Signal Comparison"
):
    with warnings.catch_warnings():
        warnings.filterwarnings("ignore", category=RuntimeWarning)
        
        t = np.arange(len(pred_signals[0]))
        
        fig, axs = plt.subplots(1, 2, figsize=(15, 10))
        
        axs[0].plot(t, pred_signals[0], label='Predicted Signal Vx')
        axs[0].set_ylabel('Amplitude')
        axs[0].legend()
        axs[0].grid(True)
        
        axs[1].plot(t, pred_signals[1], label='Predicted Signal Vy')
        axs[1].set_ylabel('Amplitude')
        axs[1].legend()
        axs[1].grid(True)
        
        axs[0].plot(t, adjusted_signals[0], label='Adjusted Signal Vx', dashes=[6, 2])
        axs[0].set_ylabel('Amplitude')
        axs[0].legend()
        axs[0].grid(True)
        
        axs[1].plot(t, adjusted_signals[1], label='Adjusted Signal Vy', dashes=[6, 2])
        axs[1].set_ylabel('Amplitude')
        axs[1].legend()
        axs[1].grid(True)
        
        axs[0].plot(t, reference_signals[0], label='Reference Signal Vx')
        axs[0].set_xlabel('Time')
        axs[0].set_ylabel('Amplitude')
        axs[0].legend()
        axs[0].grid(True)
        
        axs[1].plot(t, reference_signals[1], label='Reference Signal Vy')
        axs[1].set_xlabel('Time')
        axs[1].set_ylabel('Amplitude')
        axs[1].legend()
        axs[1].grid(True)
        
        fig.suptitle(title)
        plt.tight_layout()

        plt.savefig(os.path.join(out_dir, f'{model_name}.png'))

        if show_plot:
            plt.show()


In [7]:
model_final_results = []

test_loader = DataLoader(ScrewdrivingDataset(mode='test', **{**vars(ARGS), 'data_dir': data_dir}))
test_x = torch.cat([batch[0] for batch in test_loader], dim=0)

test_y = [batch[1].numpy() for batch in DataLoader(ScrewdrivingDataset(mode='test', **{**vars(ARGS), 'data_dir': data_dir}))]
test_y = np.concatenate(test_y, axis=0)
test_y = test_y.reshape(test_y.shape[0], test_y.shape[-1])

initial_guess = [0., 0.]

In [8]:
for idx, model_name in enumerate(model_checkpoints):
    with warnings.catch_warnings():
        warnings.filterwarnings("ignore", category=RuntimeWarning)
        
        with torch.no_grad():
            model = SindyModel(**vars(ARGS))
            model.load_state_dict(torch.load(os.path.join(model_dir, model_name)))
            model.eval()
    
            pred_y = model(test_x).numpy()
            pred_y = pred_y.reshape(pred_y.shape[0], pred_y.shape[-1]).astype(np.complex64)
    
            reference_signal_x = np.copy(test_y)[:, 0]
            target_signal_x = pred_y[:, 0]
            result_vx = minimize(signal_error, initial_guess.copy(), args=(reference_signal_x, target_signal_x), method='Nelder-Mead')
            adjusted_vx = adjust_signal(target_signal_x, *result_vx.x)
    
            reference_signal_y = np.copy(test_y)[:, 1]
            target_signal_y = pred_y[:, 1]
            result_vy = minimize(signal_error, initial_guess.copy(), args=(reference_signal_y, target_signal_y), method='Nelder-Mead')
            adjusted_vy = adjust_signal(target_signal_y, *result_vy.x)
    
            plot_signals([pred_y[:, 0], pred_y[:, 1]], [adjusted_vx, adjusted_vy], [reference_signal_x, reference_signal_y], model_name, False)
    
            final_x_error = signal_error(result_vx.x, reference_signal_x, adjusted_vx)
            final_y_error = signal_error(result_vy.x, reference_signal_y, adjusted_vy)
    
            print(f'#: {idx}, model: {model_name}, vx_err: {final_x_error}, vx_params: {result_vx.x}, vy_err: {final_y_error}, vy_params: {result_vy.x}')
            
            model_final_results.append((model_name, np.real(final_x_error), *result_vx.x, np.real(final_y_error), *result_vy.x))
            

Model: model_2024-05-09 22:01:42.786493.pth, vx err: (229.44934380265943+0j), vx_params: [0.8745272  1.46131171], vy_err: (342.50971532630604+0j), vy_params: [ 1.06841511 -0.2873906 ]
Model: model_2024-05-09 22:12:13.978245.pth, vx err: (3.8253851736924718+0j), vx_params: [ 0.11200665 -3.14791124], vy_err: (113.8819538231586+0j), vy_params: [ 0.61523123 -3.69068803]
Model: model_2024-05-09 22:14:46.276889.pth, vx err: (92.93092595504514+0j), vx_params: [-0.55642573  0.99997629], vy_err: (470.4152262616244+0j), vy_params: [-1.25200434 -1.81546371]
Model: model_2024-05-09 22:17:16.523394.pth, vx err: (274.5490615361647+0j), vx_params: [0.95675279 1.56663985], vy_err: (259.05725555606875+0j), vy_params: [ 0.92902575 -1.1754241 ]
Model: model_2024-05-09 22:19:51.606696.pth, vx err: (58.71744749405772+0j), vx_params: [-0.44238385  2.33868439], vy_err: (1.169345309904857+0j), vy_params: [ 0.06103896 -0.99995817]
Model: model_2024-05-09 22:22:31.619788.pth, vx err: (28.2230294132895+0j), vx_p

Exception ignored in: <function TransformNode.set_children.<locals>.<lambda> at 0x73e5f3956710>
Traceback (most recent call last):
  File "/home/rutvik/Desktop/screwdriving-model-sindy-rcim/venv/lib/python3.10/site-packages/matplotlib/transforms.py", line 198, in <lambda>
    self, lambda _, pop=child._parents.pop, k=id_self: pop(k))
KeyboardInterrupt: 


Model: model_2024-05-09 23:39:11.157484.pth, vx err: (159.63039744907314+0j), vx_params: [ 0.72892765 -1.65984716], vy_err: (170.53600247333193+0j), vy_params: [ 0.75392037 -0.48697323]
Model: model_2024-05-09 23:41:40.663431.pth, vx err: (26.348534336229402+0j), vx_params: [0.29579133 2.81556595], vy_err: (120.12885273394105+0j), vy_params: [-0.63166727  3.92196576]
Model: model_2024-05-09 23:44:14.038231.pth, vx err: (37.78868394439512+0j), vx_params: [-0.35468882  1.78853199], vy_err: (277.28476547920275+0j), vy_params: [-0.96164077  3.63762771]


KeyboardInterrupt: 

In [ ]:
out_file_path = os.path.join(out_dir, 'cumulative_results.csv')
np.savetxt(out_file_path, model_final_results, fmt='%s', delimiter=',')

In [ ]:
headers = ['model_name', 'vx_error', 'mean_adjustment_vx', 'time_shift_vx', 'vy_error', 'mean_adjustment_vy', 'time_shift_vy']

with open(out_file_path, 'r+') as fh:
    content = fh.read()
    fh.seek(0, 0)
    fh.write(','.join(headers) + '\n' + content)
